### Event Analysis 

In [161]:
import pandas as pd
import math
from itertools import combinations

In [162]:
### Load Data 
voteData = pd.read_csv(r'C:\Users\beneb\Desktop\plural_data\votes.csv')
groupData = pd.read_csv(r'C:\Users\beneb\Desktop\plural_data\users_to_groups.csv')
forumQuestions = pd.read_csv(r'C:\Users\beneb\Desktop\plural_data\forum_questions.csv')
groupCategories = pd.read_csv(r'C:\Users\beneb\Desktop\plural_data\group_categories.csv')
questionOptions = pd.read_csv(r'C:\Users\beneb\Desktop\plural_data\question_options.csv')

# print("voteData:", len(voteData))
# print("groupData:", len(groupData))
# print("forumQuestions:", len(forumQuestions))
# print("groupCategories:", len(groupCategories))

In [163]:
def getLatestVoteByUserAndOptionId(df, option_id):
    """
    Gets the latest vote data by users for a specified option ID.
    :param: df (pandas.DataFrame): The DataFrame containing the vote data.
    :param: option_id (str): The ID of the option for which to query vote data.
    """
    
    # Filter the DataFrame by option_id
    filtered_df = df[df['option_id'] == option_id].copy()

    # Rank each user by updated_at in descending order
    filtered_df['rank'] = filtered_df.groupby('user_id')['updated_at'].rank(method='first', ascending=False)

    # Select the latest vote for each user
    latest_votes = filtered_df[filtered_df['rank'] == 1]

    # Create a dictionary from the DataFrame
    vote_dict = latest_votes.set_index('user_id')['num_of_votes'].to_dict()

    return vote_dict

In [164]:
voteDict = getLatestVoteByUserAndOptionId(voteData, '4ad65252-e651-460c-af6b-289769a3964b')
# print(voteDict)

In [165]:
def filterZeroVotes(vote_dict):
    """
    Filters out zero votes from the dictionary.
    
    :param vote_dict: dict - A dictionary with user IDs as keys and their number of votes as values.
    :return: dict - A dictionary with zero votes removed.
    """
    
    # Filter out entries with zero votes
    filtered_dict = {user_id: votes for user_id, votes in vote_dict.items() if votes != 0}
    
    return filtered_dict

In [166]:
filteredVoteDict = filterZeroVotes(voteDict)
# print(filteredVoteDict)

In [167]:
def getGroupsByUserAndOptionId(df, numOfVotesDictionary, groupCategories):
    """
    Gets group data and creates group dictionary based on user IDs and option ID.
    :param: df (pandas.DataFrame): The DataFrame containing the group data.
    :param: numOfVotesDictionary (dict): Dictionary of user IDs and their respective number of votes.
    :param: groupCategories (list of str): Array of group category IDs.
    """

    # Filter users_to_groups DataFrame by user IDs and group category IDs
    filtered_df = df[df['user_id'].isin(numOfVotesDictionary.keys()) & df['group_category_id'].isin(groupCategories)]

    # Group by group_id and aggregate user_ids into a list
    group_array = filtered_df.groupby('group_id')['user_id'].agg(list).reset_index()

    # Convert group_array DataFrame to dictionary
    groups_dictionary = dict(zip(group_array['group_id'], group_array['user_id']))

    return groups_dictionary


In [168]:
group_dict = getGroupsByUserAndOptionId(groupData, voteDict, ['0419db52-cda8-4cf2-adb2-a881234ea1f8', 'b1862f74-458c-4eb8-9445-1ccf9ccd1712'])

# Print the resulting dictionary
# print(group_dict)

In [169]:
def create_group_memberships(groups):
    """
    Define group memberships for each participant.
    :param: groups (list of lists): a list denotes the group and contains its members. 
    :param: votes (list): number of participant's votes for a given project proposal.
    :returns (list of lists): retunrs a list of group membersips for each participant.  
    """
    memberships = {}
    for group, members in groups.items():
        for member in members:
            memberships.setdefault(member, []).append(group)
    return memberships

In [170]:
memberships = create_group_memberships(group_dict)
# print(memberships)

In [171]:
def removeDuplicateGroups(input_groups: dict[str, list[str]]) -> dict[str, list[str]]:
    ''' 
     Removes duplicate groups from the input array of groups.
     :param: inputGroups An array of groups (can contain duplicates).
    '''
    unique_groups = {}

    for group_name, users in input_groups.items():
        # Sort the user arrays to ensure order doesn't matter
        sorted_users = sorted(users)

        # Check if the sorted array is not already in unique_groups
        if not any(sorted(existing_group) == sorted_users for existing_group in unique_groups.values()):
            unique_groups[group_name] = sorted_users

    return unique_groups

In [172]:
filteredGroups = removeDuplicateGroups(group_dict)

In [173]:
def ConnectionOrientedClusterMatch(groups, votes):
    # memberships[i] is the number of groups agent i is in
    memberships = {user: sum(user in members for members in groups.values()) for user in votes.keys()}

    # friend_matrix[i][j] is the number of groups that agent i and j are both in
    friend_matrix = {
        user1: {user2: sum(user1 in members and user2 in members for members in groups.values()) for user2 in votes.keys()}
        for user1 in votes.keys()
    }

    # build up the funding amount. First, add in everyone's contributions
    qf_score = sum(votes.values())

    def K(user, h):
        if sum(friend_matrix[user][other_user] for other_user in h) > 0:
            return math.sqrt(votes[user])
        return votes[user]

    qf_score += sum(
        2 * math.sqrt(sum(K(user, groups[group1]) / memberships[user] for user in groups[group0]))
        * math.sqrt(sum(K(other_user, groups[group0]) / memberships[other_user] for other_user in groups[group1]))
        for group0, group1 in combinations(groups.keys(), 2)
    )

    qv_score = math.sqrt(qf_score)

    return qv_score

In [174]:
# Test data
groups = {
    'group0': ['user0', 'user1'],
    'group1': ['user1', 'user2', 'user3'],
    'group2': ['user0', 'user2'],
}

contributions = {
    'user0': 1,
    'user1': 2,
    'user2': 3,
    'user3': 4,
}

result_2 = ConnectionOrientedClusterMatch(filteredGroups, filteredVoteDict)
# print(result_2)


### Replication 

In [175]:
optionIds = questionOptions[questionOptions['question_id'] == '5eaea820-8b02-446c-bf17-59f6fc1799bc']['id'].tolist()
# print(len(optionIds))


In [176]:
results = []

for optionId in optionIds:
    voteDict = getLatestVoteByUserAndOptionId(voteData, optionId)
    filteredVoteDict = filterZeroVotes(voteDict)
    groupDict = getGroupsByUserAndOptionId(groupData, filteredVoteDict, ['0419db52-cda8-4cf2-adb2-a881234ea1f8', 'b1862f74-458c-4eb8-9445-1ccf9ccd1712'])
    filteredGroups = removeDuplicateGroups(groupDict)
    result = ConnectionOrientedClusterMatch(filteredGroups, filteredVoteDict)
    results.append(result)

# Sort the list in descending order
sorted_results = sorted(results, reverse=True)

# print(sorted_results)
